In [ ]:
#Hi Dr. McInnes
#I was able to make a one-click notebook. However, the output is buried in the
#code, so you may find it easier to just go through the notebook using "Run".

#With that said, we didn't have a tightly-coordinated development process, so
#in the interest of representing our respective contribution authentically,
#I've minimized the number of modifications made to our code. I hope this
#helps explain any haphazardness evident from a design perspective.

#Also, I found two bugs while assembling this notebook. I have an idea of how
#to fix them, but, again, in the interest of authenticity, I've left them
#unmodified (a softmax layer in the sigmoid MLP, and a bug in model_test()).

In [ ]:
import gensim
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import os

In [ ]:
#Jesse
def get_scores():
    os.system("python eval/evaluate.py eval/input eval/output")
    fh = open("eval/output/scores.txt")
    for line in fh.readlines():
        line = line.split(':')
        descriptor = line[0].replace("score", "match")
        if descriptor == "match":
            descriptor = "match avg"
        score = line[1]
        print(descriptor+": "+score)
    fh.close()

#**Data Processing**

In [ ]:
# Author : Vasanthi tarigopula

f = open("train.txt", "r")
lines=f.readlines()

sent=[[] for i in lines]                    ### initializing list
probs=[[] for i in lines]                   ### Intializing list 
ids = [[] for i in lines]

j=0
word=''
prob=''
for i in range(0,len(lines)):               ### loop on each line of file
    cols=lines[i].strip().split('\t')         ### spliting line on the basis of tab
    if (len(cols)>2):                         ### if line has data
        sent[j].append(cols[1].lower())                 ### Appending word in list named sent
        probs[j].append(cols[4])                ### Appending probs in list named probs
        ids[j].append(cols[0])
    else:
        j=j+1
        
sent = [x for x in sent if x != []]
probs = [x for x in probs if x != []]
ids = [x for x in ids if x != []]

##**Training Word2Vec to learn embeddings**

In [ ]:
# Author : Vasanthi Tarigopula
# derived from https://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.XZyzt5NKjOQ
max_features = 150
max_words = 38
# instantiating Word2Vec to train the model by passing the list of lists
# build vocabulary and train model

model = gensim.models.Word2Vec(
    sent,
    size=max_features,
    window=10,
    min_count=1,
    workers=10,
    iter=30)

**Function to find embeddings**

In [ ]:
# Author : Vasanthi Tarigopula
'''  *******************************************************************
       Returns Combined Embeddings of Title and Description of an Issue
     *******************************************************************  '''

def get_sentence_embeddings(sentence_tokens):
  
    #to store embeddings of all the words in the sentence    
    sent_embeddings= np.zeros([max_words,max_features ])
    i = 0
    #iterating over all the words in the title one by one
    for word in sentence_tokens:
        
        sent_embeddings[i] = np.array(model[word])  #embedding of a single word
        i=i+1
        
    if len(sentence_tokens) < max_words:
      for i in range(len(sentence_tokens), max_words):
        sent_embeddings[i] = np.zeros(max_features)
        i = i +1
         
    #mean embedding of all the words in title      
    #mean_embedings = np.mean(np.array(sent_embeddings),axis=0)  
    
    return np.array(sent_embeddings)

In [ ]:
# Author : Vasanthi Tarigopula
data={}
X = []
Y= []
for i in range(len(sent)):
    
    sentence_embeddings = get_sentence_embeddings(sent[i])
    #print ('Sent[0]',sentence_embeddings[0])
    tmp = []
    for prob in probs[i]:
      tmp.append(float(prob))  
    if len(tmp) < 38:
      for j in range(len(tmp), 38):
        tmp.append(float(0))
    X.append(sentence_embeddings)
    Y.append(np.array(tmp))
   
    data[i]={"IDs":ids[i], "sentence":sent[i], "embedding":sentence_embeddings, "probs":probs[i]}  #initializing empty list at current dictionary index 

X = np.array(X)
Y = np.array(Y)

##**Declaring the BiLSTM Model**

In [ ]:
# Author : Vasanthi tarigopula
# derived from https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding,Bidirectional
from keras.layers import LSTM

max_features = 150
SEQUENCE_LEN = max_words
lstm_model = Sequential()
lstm_model.add(Bidirectional(LSTM(128), input_shape=(SEQUENCE_LEN, max_features)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(max_words, activation='sigmoid'))

lstm_model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['mae'])

print(lstm_model.summary())

lstm_model.fit(X[:1650,:,:],Y[:1650,:], batch_size=10, epochs=10)
#score = model.evaluate(x_test, y_test, batch_size=16)

**Outputting results for model comparison**

In [ ]:
#from https://github.com/RiTUAL-UH/SemEval2020_Task10_Emphasis_Selection/blob/master/try_it/How_to_Read_data_and_Write_results.py

def write_results(word_id_lsts, words_lsts, e_freq_lsts, write_to):
    """
    This function writes results in the format.
    :param word_id_lsts: list of word_ids
    :param words_lsts: list of words
    :param e_freq_lsts: lists of emphasis probabilities
    :param write_to: writing directory
    :return:
    """


    with open(write_to, 'w') as out:
        sentence_id=""
        # a loop on sentences:
        for i in range(len(words_lsts)):
            # a loop on words in a sentence:
            for j in range(len(words_lsts[i])):
                # writing:
                if sentence_id ==i:
                    to_write = "{}\t{}\t{}\t".format(word_id_lsts[i][j], words_lsts[i][j], e_freq_lsts[i][j])
                    out.write(to_write + "\n")
                else:
                    out.write("\n")
                    to_write = "{}\t{}\t{}\t".format(word_id_lsts[i][j], words_lsts[i][j], e_freq_lsts[i][j])
                    out.write(to_write + "\n")
                    sentence_id = i
        out.write("\n")
        out.close()

In [ ]:
#Jesse
word_id_lsts = []
words_lsts = []
e_freq_lsts = []

for i in range(1650,len(data)):
    predicted_probs = lstm_model.predict(X[i:i+1,:,:])[0]
    word_id_lsts.append(data[i]['IDs'])
    words_lsts.append(data[i]['sentence'])
    e_freq_lsts.append(predicted_probs)
    
write_results(word_id_lsts, words_lsts, e_freq_lsts, 'eval/input/res/submission.txt')

print("BiLSTM model")
get_scores()

****Start CNN & MLP code****

**Defining helper functions**

In [ ]:
#Jesse
from gensim.models import KeyedVectors
import random
import numpy as np

#adapted from Vasanthi's code
def data_loader(filename):
    f = open(filename, "r")
    lines=f.readlines()

    sent=[]                    ### initializing list
    probs=[]                   ### Intializing list 
    ids=[]
    data = []
    
    for i in range(0,len(lines)):               ### loop on each line of file
        cols=lines[i].strip().split('\t')         ### spliting line on the basis of tab
        if (len(cols)>2):                         ### if line has data
            sent.append(cols[1].lower())                 ### Appending word in list named sent
            probs.append(float(cols[4]))                ### Appending probs in list named probs
            ids.append(cols[0])
        else:
            if len(sent) > 0:
                data.append(DataItem(sent,probs,ids))
            sent = []
            probs = []
            ids = []

    return data

#Jesse
def get_max_length(dataset):
    longest = 0
    
    for item in dataset:
        if len(item.word_seq) > longest:
            longest = len(item.word_seq)
            
    return longest

#Jesse
#def two_split(dataset, split_point):
    #temp = dataset[:]
    #random.shuffle(temp)
    #return temp[0:int(len(temp)*split_point)], temp[int(len(temp)*split_point):]

#Jesse
def two_split(dataset, split_point):
    temp = dataset
    return temp[:1650], temp[1650:]

#Jesse
class Sentence2Matrix:
    def __init__(self, filename):
        self.word_model = KeyedVectors.load(filename, mmap='r')
        
    def __getitem__(self, word_seq):
            temp = [list(self.word_model[word]) for word in word_seq]
            return temp

#Jesse
class DataItem:
    def __init__(self, word_seq, prob_seq, id_seq):
        self.word_seq = word_seq
        self.prob_seq = prob_seq
        self.id_seq = id_seq

**Load and split data**

In [ ]:
#Jesse
dataset = data_loader("train.txt")
max_length = get_max_length(dataset)
train_set, test_set = two_split(dataset, 0.6)

mtrxr = Sentence2Matrix("w2v.vec")
train_x = [mtrxr[i.word_seq] for i in train_set]
train_y = [i.prob_seq for i in train_set]

test_x = [mtrxr[i.word_seq] for i in test_set]
test_y = [i.prob_seq for i in test_set]

**Pad input data**

In [ ]:
#Jesse
def pad_input(matrix, max_length):
    embedding_length = len(matrix[0])
    empty_vector = [0.0 for i in range(embedding_length)]

    if len(matrix) < max_length:
        for i in range(max_length-len(matrix)):
            matrix.append(empty_vector[:])
            
    return matrix
    
train_x = np.array([np.array([pad_input(i, max_length)]) for i in train_x])
test_x = np.array([np.array([pad_input(i, max_length)]) for i in test_x])

**Pad output data**

In [ ]:
#Jesse
def pad_output(prob_seq, max_length):
    if len(prob_seq) < max_length:
        for i in range(max_length-len(prob_seq)):
            prob_seq.append(0.0)
    return prob_seq
            
train_y = np.array([np.array(pad_output(i, max_length)) for i in train_y])
test_y = np.array([np.array(pad_output(i, max_length)) for i in test_y])

**Define model testing function**

In [ ]:
#Jesse
def model_test(model, test_items):
    word_id_lsts = []
    words_lsts = []
    e_freq_lsts = []
    predicted_probs = model.predict(test_items)

    for i in range(len(test_items)):
        data_item = test_set[i]

        word_id_lsts.append(data_item.id_seq)
        words_lsts.append(data_item.word_seq)
        e_freq_lsts.append(predicted_probs[i])

    write_results(word_id_lsts, words_lsts, e_freq_lsts, 'eval/input/res/submission.txt')

**Sigmoid CNN**

In [ ]:
#Jesse
#derived from https://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/ and Vasanthi's code
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

feature_length = len(train_x[0][0][0])

model = Sequential()

#input_shape = (img_x, img_y, 1)
#kernel_h, kernel_w = self.kernel_size

#was having issues due to max pooling 2d layer trying to iterate over what was effectively a 1D convolution
#max pooling 1D layer wouldn't work due to formatting issues
#might work if I could reshape the (pseudo-1D) "2D convolution" into an actual 1D convolution

model.add(Conv2D(filters=16, kernel_size=(3,feature_length-1), data_format='channels_first', input_shape=(1, max_length, feature_length)))
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Flatten())

model.add(Dropout(0.15))

model.add(Dense(100, activation="sigmoid"))
model.add(Dense(50, activation="sigmoid"))
model.add(Dense(max_length, activation="sigmoid"))

model.compile(loss="mean_squared_error", 
              optimizer="rmsprop",
              metrics=['mae'])

model.fit(train_x, train_y, batch_size=10, epochs=10)
model_test(model, test_x)

print("Sigmoid CNN")
get_scores()

**Softmax CNN**

In [ ]:
#Jesse
#derived from https://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/ and Vasanthi's code
feature_length = len(train_x[0][0][0])

model = Sequential()

#input_shape = (img_x, img_y, 1)
#kernel_h, kernel_w = self.kernel_size

#was having issues due to max pooling 2d layer trying to iterate over what was effectively a 1D convolution
#max pooling 1D layer wouldn't work due to formatting issues
#might work if I could reshape the (pseudo-1D) "2D convolution" into an actual 1D convolution

model.add(Conv2D(filters=16, kernel_size=(3,feature_length-1), data_format='channels_first', input_shape=(1, max_length, feature_length)))
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Flatten())

model.add(Dropout(0.15))

model.add(Dense(100, activation="sigmoid"))
model.add(Dense(50, activation="sigmoid"))
model.add(Dense(max_length, activation="softmax"))

model.compile(loss="mean_squared_error", 
              optimizer="rmsprop",
              metrics=['mae'])

model.fit(train_x, train_y, batch_size=10, epochs=10)
model_test(model, test_x)
print("Softmax CNN")
get_scores()

**Sigmoid MLP**

In [ ]:
#Original code by Vasanthi, edits by Jesse
from keras.layers import Softmax

model = Sequential()
model.add(Flatten())
model.add(Dense(1000, input_dim=38*150, activation='sigmoid'))
model.add(Dense(240, activation='sigmoid'))
model.add(Dense(38, activation='sigmoid'))
model.add(Softmax())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['MAE'])
model.fit(X[:1650,:,:],Y[:1650,:], batch_size=10, epochs=5)

word_id_lsts = []
words_lsts = []
e_freq_lsts = []

for i in range(1650,len(data)):
    predicted_probs = model.predict(X[i:i+1,:,:])[0]
    word_id_lsts.append(data[i]['IDs'])
    words_lsts.append(data[i]['sentence'])
    e_freq_lsts.append(predicted_probs)
    
write_results(word_id_lsts, words_lsts, e_freq_lsts, 'eval/input/res/submission.txt')

print("Sigmoid MLP model")
get_scores()


**Softmax MLP**

In [ ]:
#Original code by Vasanthi, edits by Jesse
model = Sequential()
model.add(Flatten())
model.add(Dense(1000, input_dim=38*150, activation='sigmoid'))
model.add(Dense(240, activation='sigmoid'))
model.add(Dense(38, activation='softmax'))
model.add(Softmax())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['MAE'])

model.fit(X[:1650,:,:],Y[:1650,:], batch_size=10, epochs=5)

word_id_lsts = []
words_lsts = []
e_freq_lsts = []

for i in range(1650,len(data)):
    predicted_probs = model.predict(X[i:i+1,:,:])[0]
    word_id_lsts.append(data[i]['IDs'])
    words_lsts.append(data[i]['sentence'])
    e_freq_lsts.append(predicted_probs)
    
write_results(word_id_lsts, words_lsts, e_freq_lsts, 'eval/input/res/submission.txt')

print("Softmax MLP model")
get_scores()
